# Chapter 5: Building Crystal Structures

## Learning Objectives
- Build crystal structures from lattice + basis
- Implement common crystal structures (FCC, BCC, diamond, etc.)
- Understand space groups and Wyckoff positions
- Create supercells and manipulate crystal structures

---

## 5.1 Crystal Structure = Lattice + Basis

A crystal structure consists of:
1. **Lattice**: The periodic framework (points in space)
2. **Basis**: The atom(s) associated with each lattice point

To build a crystal:
1. Define the unit cell (lattice vectors)
2. Place atoms at positions within the cell (basis)
3. The infinite crystal is generated by translating the basis by all lattice vectors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass, field

@dataclass
class LatticeParameters:
    a: float
    b: float
    c: float
    alpha: float = 90.0
    beta: float = 90.0
    gamma: float = 90.0

def lattice_vectors_from_parameters(params: LatticeParameters) -> np.ndarray:
    alpha = np.radians(params.alpha)
    beta = np.radians(params.beta)
    gamma = np.radians(params.gamma)
    
    a_vec = np.array([params.a, 0, 0])
    bx = params.b * np.cos(gamma)
    by = params.b * np.sin(gamma)
    b_vec = np.array([bx, by, 0])
    
    cx = params.c * np.cos(beta)
    cy = params.c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma)
    cz = np.sqrt(max(0, params.c**2 - cx**2 - cy**2))
    c_vec = np.array([cx, cy, cz])
    
    return np.array([a_vec, b_vec, c_vec])

class Crystal:
    """A crystal structure with unit cell and atomic basis."""
    
    # Element properties for visualization
    ELEMENTS = {
        'H':  {'Z': 1,  'color': 'white', 'radius': 0.31},
        'C':  {'Z': 6,  'color': 'gray', 'radius': 0.76},
        'N':  {'Z': 7,  'color': 'blue', 'radius': 0.71},
        'O':  {'Z': 8,  'color': 'red', 'radius': 0.66},
        'Na': {'Z': 11, 'color': 'violet', 'radius': 1.66},
        'Mg': {'Z': 12, 'color': 'green', 'radius': 1.41},
        'Al': {'Z': 13, 'color': 'lightgray', 'radius': 1.21},
        'Si': {'Z': 14, 'color': 'gold', 'radius': 1.11},
        'Cl': {'Z': 17, 'color': 'lime', 'radius': 1.02},
        'Ca': {'Z': 20, 'color': 'darkgreen', 'radius': 1.76},
        'Ti': {'Z': 22, 'color': 'silver', 'radius': 1.60},
        'Fe': {'Z': 26, 'color': 'orange', 'radius': 1.32},
        'Cu': {'Z': 29, 'color': 'brown', 'radius': 1.40},
        'Zn': {'Z': 30, 'color': 'slategray', 'radius': 1.22},
        'Ga': {'Z': 31, 'color': 'pink', 'radius': 1.22},
        'As': {'Z': 33, 'color': 'purple', 'radius': 1.19},
        'Sr': {'Z': 38, 'color': 'yellowgreen', 'radius': 2.00},
        'Au': {'Z': 79, 'color': 'gold', 'radius': 1.36},
    }
    
    def __init__(self, name: str, lattice_vectors: np.ndarray):
        """Initialize a crystal structure.
        
        Args:
            name: Name of the crystal
            lattice_vectors: 3x3 matrix of lattice vectors (rows are a, b, c)
        """
        self.name = name
        self.lattice_vectors = np.array(lattice_vectors, dtype=float)
        self._inv_lattice = np.linalg.inv(self.lattice_vectors)
        
        # Basis: list of (symbol, fractional_coords)
        self.basis: List[Tuple[str, np.ndarray]] = []
    
    @classmethod
    def from_parameters(cls, name: str, params: LatticeParameters) -> 'Crystal':
        """Create crystal from lattice parameters."""
        vectors = lattice_vectors_from_parameters(params)
        return cls(name, vectors)
    
    def add_atom(self, symbol: str, fractional_coords: np.ndarray) -> None:
        """Add an atom to the basis.
        
        Args:
            symbol: Element symbol
            fractional_coords: Position in fractional coordinates [fa, fb, fc]
        """
        self.basis.append((symbol, np.array(fractional_coords, dtype=float)))
    
    def add_atoms(self, symbol: str, positions: List[np.ndarray]) -> None:
        """Add multiple atoms of the same element."""
        for pos in positions:
            self.add_atom(symbol, pos)
    
    @property
    def volume(self) -> float:
        """Volume of unit cell in Å³."""
        return abs(np.linalg.det(self.lattice_vectors))
    
    @property
    def n_atoms(self) -> int:
        """Number of atoms in the basis."""
        return len(self.basis)
    
    def fractional_to_cartesian(self, frac: np.ndarray) -> np.ndarray:
        """Convert fractional to Cartesian coordinates."""
        return frac @ self.lattice_vectors
    
    def cartesian_to_fractional(self, cart: np.ndarray) -> np.ndarray:
        """Convert Cartesian to fractional coordinates."""
        return cart @ self._inv_lattice
    
    def get_cartesian_positions(self) -> Tuple[List[str], np.ndarray]:
        """Get all atomic positions in Cartesian coordinates."""
        symbols = [s for s, _ in self.basis]
        frac_positions = np.array([p for _, p in self.basis])
        cart_positions = self.fractional_to_cartesian(frac_positions)
        return symbols, cart_positions
    
    def make_supercell(self, na: int, nb: int, nc: int) -> 'Crystal':
        """Create a supercell by replicating the unit cell.
        
        Args:
            na, nb, nc: Number of repetitions along each lattice vector
        
        Returns:
            New Crystal with expanded cell
        """
        # New lattice vectors
        new_vectors = self.lattice_vectors * np.array([[na], [nb], [nc]])
        supercell = Crystal(f"{self.name}_{na}x{nb}x{nc}", new_vectors)
        
        # Replicate atoms
        for symbol, frac in self.basis:
            for i in range(na):
                for j in range(nb):
                    for k in range(nc):
                        new_frac = (frac + np.array([i, j, k])) / np.array([na, nb, nc])
                        supercell.add_atom(symbol, new_frac)
        
        return supercell
    
    def __repr__(self) -> str:
        return f"Crystal('{self.name}', {self.n_atoms} atoms, V={self.volume:.2f} Å³)"

print("Crystal class defined!")

## 5.2 Common Crystal Structures

Let's build the most common crystal structures in materials science.

### Simple Cubic (SC)
- 1 atom per unit cell
- Coordination number: 6
- Examples: Polonium

In [ ]:
def build_simple_cubic(element: str, a: float) -> Crystal:
    """Build a simple cubic crystal.
    
    Args:
        element: Element symbol
        a: Lattice constant in Å
    """
    crystal = Crystal.from_parameters(f"{element}_SC", 
                                       LatticeParameters(a, a, a, 90, 90, 90))
    crystal.add_atom(element, [0, 0, 0])
    return crystal

# Example: Polonium (only element with SC structure at room temp)
po_sc = build_simple_cubic('Po', 3.35)  # Actually Po has a = 3.35 Å
print(po_sc)

### Body-Centered Cubic (BCC)
- 2 atoms per unit cell
- Coordination number: 8
- Examples: Fe (α), W, Cr, Mo, Na, K

In [ ]:
def build_bcc(element: str, a: float) -> Crystal:
    """Build a body-centered cubic crystal."""
    crystal = Crystal.from_parameters(f"{element}_BCC",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    # Corner atom
    crystal.add_atom(element, [0, 0, 0])
    # Body center atom
    crystal.add_atom(element, [0.5, 0.5, 0.5])
    return crystal

# α-Iron (ferrite)
fe_bcc = build_bcc('Fe', 2.87)
print(fe_bcc)

# Tungsten
w_bcc = build_bcc('W', 3.16)
print(w_bcc)

### Face-Centered Cubic (FCC)
- 4 atoms per unit cell
- Coordination number: 12
- Examples: Cu, Ag, Au, Al, Ni, Pt

In [ ]:
def build_fcc(element: str, a: float) -> Crystal:
    """Build a face-centered cubic crystal."""
    crystal = Crystal.from_parameters(f"{element}_FCC",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    # FCC positions:
    # Corner: (0,0,0)
    # Face centers: (0.5,0.5,0), (0.5,0,0.5), (0,0.5,0.5)
    fcc_positions = [
        [0, 0, 0],
        [0.5, 0.5, 0],
        [0.5, 0, 0.5],
        [0, 0.5, 0.5]
    ]
    crystal.add_atoms(element, fcc_positions)
    return crystal

# Copper
cu_fcc = build_fcc('Cu', 3.615)
print(cu_fcc)

# Gold
au_fcc = build_fcc('Au', 4.078)
print(au_fcc)

### Hexagonal Close-Packed (HCP)
- 2 atoms per unit cell
- Coordination number: 12
- Examples: Mg, Zn, Ti, Co

In [ ]:
def build_hcp(element: str, a: float, c: float = None) -> Crystal:
    """Build a hexagonal close-packed crystal.
    
    Args:
        element: Element symbol
        a: Lattice constant in Å
        c: c-axis length (default: ideal c/a = sqrt(8/3) ≈ 1.633)
    """
    if c is None:
        c = a * np.sqrt(8/3)  # Ideal c/a ratio
    
    crystal = Crystal.from_parameters(f"{element}_HCP",
                                       LatticeParameters(a, a, c, 90, 90, 120))
    # HCP positions in hexagonal cell:
    # (0, 0, 0) and (1/3, 2/3, 1/2)
    crystal.add_atom(element, [0, 0, 0])
    crystal.add_atom(element, [1/3, 2/3, 0.5])
    return crystal

# Magnesium (nearly ideal HCP)
mg_hcp = build_hcp('Mg', 3.21, 5.21)  # c/a = 1.624
print(mg_hcp)

# Titanium
ti_hcp = build_hcp('Ti', 2.95, 4.68)  # c/a = 1.587
print(ti_hcp)

### Diamond Cubic
- 8 atoms per unit cell
- Coordination number: 4 (tetrahedral)
- Examples: C (diamond), Si, Ge, α-Sn

In [ ]:
def build_diamond(element: str, a: float) -> Crystal:
    """Build a diamond cubic crystal.
    
    Diamond = FCC + FCC shifted by (1/4, 1/4, 1/4)
    """
    crystal = Crystal.from_parameters(f"{element}_diamond",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    
    # FCC positions
    fcc = np.array([
        [0, 0, 0],
        [0.5, 0.5, 0],
        [0.5, 0, 0.5],
        [0, 0.5, 0.5]
    ])
    
    # Diamond = FCC + FCC shifted by (1/4, 1/4, 1/4)
    shift = np.array([0.25, 0.25, 0.25])
    
    crystal.add_atoms(element, fcc)
    crystal.add_atoms(element, fcc + shift)
    
    return crystal

# Silicon
si_diamond = build_diamond('Si', 5.431)
print(si_diamond)

# Verify: 8 atoms per unit cell
print(f"Atoms in unit cell: {si_diamond.n_atoms}")

### Rock Salt (NaCl) Structure
- 8 atoms per unit cell (4 Na + 4 Cl)
- Examples: NaCl, MgO, FeO

In [ ]:
def build_rocksalt(element1: str, element2: str, a: float) -> Crystal:
    """Build a rock salt (NaCl) structure.
    
    Rock salt = two interpenetrating FCC lattices
    """
    crystal = Crystal.from_parameters(f"{element1}{element2}_rocksalt",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    
    # FCC positions for element 1
    fcc1 = [
        [0, 0, 0],
        [0.5, 0.5, 0],
        [0.5, 0, 0.5],
        [0, 0.5, 0.5]
    ]
    
    # FCC positions for element 2 (shifted by 1/2 along one axis)
    fcc2 = [
        [0.5, 0, 0],
        [0, 0.5, 0],
        [0, 0, 0.5],
        [0.5, 0.5, 0.5]
    ]
    
    crystal.add_atoms(element1, fcc1)
    crystal.add_atoms(element2, fcc2)
    
    return crystal

# NaCl
nacl = build_rocksalt('Na', 'Cl', 5.64)
print(nacl)

# MgO
mgo = build_rocksalt('Mg', 'O', 4.21)
print(mgo)

### Zinc Blende (Sphalerite) Structure
- 8 atoms per unit cell (4 Zn + 4 S)
- Tetrahedral coordination
- Examples: ZnS, GaAs, InP, diamond (if both elements same)

In [ ]:
def build_zincblende(element1: str, element2: str, a: float) -> Crystal:
    """Build a zinc blende structure.
    
    Zinc blende = two interpenetrating FCC lattices shifted by (1/4, 1/4, 1/4)
    (Same as diamond, but with two different elements)
    """
    crystal = Crystal.from_parameters(f"{element1}{element2}_zincblende",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    
    fcc = [
        [0, 0, 0],
        [0.5, 0.5, 0],
        [0.5, 0, 0.5],
        [0, 0.5, 0.5]
    ]
    
    shift = np.array([0.25, 0.25, 0.25])
    
    crystal.add_atoms(element1, fcc)
    crystal.add_atoms(element2, [np.array(p) + shift for p in fcc])
    
    return crystal

# GaAs (semiconductor)
gaas = build_zincblende('Ga', 'As', 5.653)
print(gaas)

### Perovskite (ABX₃) Structure
- 5 atoms per unit cell
- A at corners, B at body center, X at face centers
- Examples: CaTiO₃, BaTiO₃, SrTiO₃

In [ ]:
def build_perovskite(A: str, B: str, X: str, a: float) -> Crystal:
    """Build a cubic perovskite (ABX3) structure.
    
    Standard orientation:
    - A at corners (0,0,0)
    - B at body center (0.5, 0.5, 0.5)
    - X at face centers
    """
    crystal = Crystal.from_parameters(f"{A}{B}{X}3_perovskite",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    
    # A at corner
    crystal.add_atom(A, [0, 0, 0])
    
    # B at body center
    crystal.add_atom(B, [0.5, 0.5, 0.5])
    
    # X at face centers (3 per unit cell)
    crystal.add_atom(X, [0.5, 0.5, 0])
    crystal.add_atom(X, [0.5, 0, 0.5])
    crystal.add_atom(X, [0, 0.5, 0.5])
    
    return crystal

# SrTiO3
srtio3 = build_perovskite('Sr', 'Ti', 'O', 3.905)
print(srtio3)

## 5.3 Visualizing Crystal Structures

In [ ]:
def plot_crystal(crystal: Crystal, supercell: Tuple[int,int,int] = (1,1,1),
                 show_cell: bool = True, ax: plt.Axes = None) -> plt.Axes:
    """Plot a crystal structure."""
    if ax is None:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
    
    # Make supercell if needed
    if supercell != (1, 1, 1):
        struct = crystal.make_supercell(*supercell)
    else:
        struct = crystal
    
    symbols, positions = struct.get_cartesian_positions()
    
    # Plot atoms by element
    unique_symbols = list(set(symbols))
    for elem in unique_symbols:
        mask = [s == elem for s in symbols]
        elem_pos = positions[mask]
        
        props = Crystal.ELEMENTS.get(elem, {'color': 'gray', 'radius': 1.0})
        ax.scatter(elem_pos[:, 0], elem_pos[:, 1], elem_pos[:, 2],
                   s=props['radius']*200, c=props['color'], 
                   edgecolors='black', label=elem, alpha=0.9)
    
    # Draw unit cell
    if show_cell:
        a, b, c = struct.lattice_vectors
        origin = np.array([0, 0, 0])
        
        # Vertices
        verts = np.array([
            origin, a, a+b, b, c, a+c, a+b+c, b+c
        ])
        
        # Edges
        edges = [(0,1), (1,2), (2,3), (3,0),
                 (4,5), (5,6), (6,7), (7,4),
                 (0,4), (1,5), (2,6), (3,7)]
        
        for i, j in edges:
            ax.plot3D(*zip(verts[i], verts[j]), 'k-', alpha=0.5, linewidth=1)
    
    ax.set_xlabel('X (Å)')
    ax.set_ylabel('Y (Å)')
    ax.set_zlabel('Z (Å)')
    ax.legend(loc='upper left')
    ax.set_title(struct.name)
    
    return ax

# Visualize different structures
fig = plt.figure(figsize=(15, 10))

structures = [
    (cu_fcc, "FCC Copper"),
    (fe_bcc, "BCC Iron"),
    (si_diamond, "Diamond Silicon"),
    (nacl, "Rock Salt NaCl"),
    (gaas, "Zinc Blende GaAs"),
    (srtio3, "Perovskite SrTiO3")
]

for i, (struct, title) in enumerate(structures):
    ax = fig.add_subplot(2, 3, i+1, projection='3d')
    plot_crystal(struct, ax=ax)
    ax.set_title(title)

plt.tight_layout()
plt.show()

## 5.4 Supercells and Periodic Images

In [ ]:
# Create and visualize a supercell
cu_supercell = cu_fcc.make_supercell(3, 3, 3)
print(f"Supercell: {cu_supercell}")
print(f"Atoms: {cu_supercell.n_atoms} (expected: 4 × 27 = 108)")

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
plot_crystal(cu_supercell, ax=ax)
ax.set_title('3×3×3 FCC Copper Supercell')
plt.show()

## 5.5 Space Groups and Wyckoff Positions

**Space groups** combine point symmetry with translations. There are 230 space groups in 3D.

**Wyckoff positions** are symmetry-equivalent positions within a unit cell.

For example, in the Fm-3m space group (FCC metals):
- 4a position: (0,0,0) - generates all FCC lattice points
- 4b position: (0.5, 0.5, 0.5) - body centers

Understanding Wyckoff positions reduces the information needed to describe a structure.

In [ ]:
# Wyckoff positions for common space groups
WYCKOFF_POSITIONS = {
    'Fm-3m': {  # FCC metals, NaCl
        '4a': [[0, 0, 0]],
        '4b': [[0.5, 0.5, 0.5]],
        '8c': [[0.25, 0.25, 0.25], [0.75, 0.75, 0.75]],
        '24d': [[0, 0.25, 0.25]],  # Partial list
    },
    'Im-3m': {  # BCC metals
        '2a': [[0, 0, 0]],
        '6b': [[0, 0.5, 0.5]],
    },
    'Fd-3m': {  # Diamond, spinel
        '8a': [[0, 0, 0], [0.25, 0.25, 0.25]],
        '16c': [[0.125, 0.125, 0.125]],
    },
    'Pm-3m': {  # Simple cubic, perovskite
        '1a': [[0, 0, 0]],
        '1b': [[0.5, 0.5, 0.5]],
        '3c': [[0, 0.5, 0.5]],
        '3d': [[0.5, 0, 0]],
    }
}

def expand_wyckoff(position: list, space_group: str) -> list:
    """Expand a Wyckoff position to all equivalent positions.
    
    This is a simplified version - full implementation requires
    the complete symmetry operations of the space group.
    """
    # For FCC (Fm-3m), the symmetry operations include face-centering
    if space_group == 'Fm-3m':
        centering = [[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]]
        expanded = []
        for pos in position:
            for c in centering:
                new_pos = (np.array(pos) + np.array(c)) % 1.0
                # Check if already in list
                is_dup = False
                for existing in expanded:
                    if np.allclose(new_pos, existing, atol=1e-6):
                        is_dup = True
                        break
                if not is_dup:
                    expanded.append(new_pos.tolist())
        return expanded
    else:
        return position

# Example: Build FCC copper using Wyckoff position
def build_from_wyckoff(element: str, a: float, space_group: str, 
                        wyckoff_site: str) -> Crystal:
    """Build crystal from space group and Wyckoff position."""
    crystal = Crystal.from_parameters(f"{element}_{space_group}",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    
    base_positions = WYCKOFF_POSITIONS[space_group][wyckoff_site]
    all_positions = expand_wyckoff(base_positions, space_group)
    
    crystal.add_atoms(element, all_positions)
    return crystal

# Build FCC copper using 4a Wyckoff position
cu_wyckoff = build_from_wyckoff('Cu', 3.615, 'Fm-3m', '4a')
print(f"Cu from Wyckoff 4a: {cu_wyckoff.n_atoms} atoms")
print("Positions:")
for sym, pos in cu_wyckoff.basis:
    print(f"  {sym}: {pos}")

---

## Practice Exercises

### Exercise 5.1: Build Wurtzite Structure
Wurtzite is the hexagonal polymorph of ZnS. Build it.

In [ ]:
# YOUR CODE HERE
def build_wurtzite(element1: str, element2: str, a: float, c: float) -> Crystal:
    """Build a wurtzite structure.
    
    Wurtzite has 4 atoms per unit cell (2 of each type).
    Positions:
    - Element1: (0, 0, 0), (1/3, 2/3, 1/2)
    - Element2: (0, 0, u), (1/3, 2/3, 1/2+u) where u ≈ 3/8
    """
    # TODO: Implement
    pass

### Exercise 5.2: Coordination Number
Write a function to calculate the coordination number (number of nearest neighbors) for each atom in a crystal.

In [ ]:
# YOUR CODE HERE
def coordination_number(crystal: Crystal, cutoff: float = None) -> Dict[int, int]:
    """Calculate coordination number for atoms in a crystal.
    
    Args:
        crystal: Crystal structure
        cutoff: Maximum distance to consider as "bonded"
                If None, estimate from nearest neighbor distance
    
    Returns:
        Dictionary mapping atom index to coordination number
    """
    # TODO: Implement (remember to consider periodic images!)
    pass

### Exercise 5.3: Fluorite Structure
Build the fluorite (CaF₂) structure.

In [ ]:
# YOUR CODE HERE
def build_fluorite(A: str, X: str, a: float) -> Crystal:
    """Build fluorite (AX2) structure.
    
    Fluorite structure:
    - A atoms on FCC lattice
    - X atoms at all tetrahedral sites (8 per unit cell)
    """
    # TODO: Implement
    pass

---

## Key Takeaways

1. **Crystal = Lattice + Basis**: Keep these concepts separate
2. **Common structures**: FCC, BCC, HCP, diamond, rock salt, perovskite
3. **Supercells** are useful for defects, surfaces, and large-scale simulations
4. **Space groups and Wyckoff positions** provide a compact representation
5. **Fractional coordinates** are natural for crystals

## Next Chapter Preview

In Chapter 6, we'll explore **Miller Indices and Crystal Planes**:
- Notation for crystal planes and directions
- Calculating d-spacing
- Visualizing crystal planes